What is this about!?

What are tensor-compilers for?
Noticing structure in operations and mapping that onto hardware efficiently?

https://research.fb.com/announcing-tensor-comprehensions/
http://pytorch.org/2018/03/05/tensor-comprehensions.html



[Tensor Comprehension](https://arxiv.org/abs/1802.04730)

* but half the time a single op would be more efficient if it what had come before and what was coming after. so efficient ops is nice, but ...? (is this already handled by traditional compilers?)
* polyhedral intermediate representation
* abstraction without regret
* how much smarter can we make the autotuner? when should it be used?

***

still doesnt get me what i want!? WANT;

* if I have some tensor network representation, I dont want to construct the entire tensor, instead I want to do the ops on the cores/arms.
* keep an online estimate of the total cost of operations, spend compute on finding new faster kernels. 
* ability to take in more context. 
* given some arbitrary tensor operation/representation, find efficient kernels.
* ?

***

https://www.reservoir.com/wp-content/uploads/2017/08/RStreamTF.pdf


In [1]:
import tensor_comprehensions as tc
import torch
lang = """
def tensordot(float(N, C1, C2, H, W) I0, float(N, C2, C3, H, W) I1) -> (O) {
    O(n, c1, c3, h, w) +=! I0(n, c1, c2, h, w) * I1(n, c2, c3, h, w)
}
"""
N, C1, C2, C3, H, W = 32, 512, 8, 2, 28, 28
tensordot = tc.define(lang, name="tensordot")
I0, I1 = torch.randn(N, C1, C2, H, W).cuda(), torch.randn(N, C2, C3, H, W).cuda()
best_options = tensordot.autotune(I0, I1, cache=True)
out = tensordot(I0, I1, options=best_options)

RuntimeError: cuda runtime error (30) : unknown error at /opt/conda/conda-bld/pytorch_1518243271935/work/torch/lib/THC/THCGeneral.c:70

The auto tune function is interesting. 
Keep track of how often a fn is called. Maybe autotune it?

We have some fn, $f_{\theta}(x)$, but we get to choose how to represent the parameters ($\theta$), that is to say that we parameterise the parameters? $\theta_{\phi}$. After selecting a different representation, we can define a new op and autotune it.

Want the representation is a function of the loss q.r.t those parameters and their speed/efficiency. 

* l = loss of the parameters. (how do you  caluclate this?)
* t = time for the
* 

$$
r = g(l, t, m)
$$


Choosing a representation is independent of finding a fsat way to compute it. No? Cos we want to pick representations that are both fast and accurate?!



In [ ]:
# WANT to be able to simultaneously optimise;
# parameters,
# their representation,
# and their compilation.

# for starters let's consider each in isolation.
# want los churn? changing the representation shouldnt effect loss much?


# in the inference case. 
# we have frozen theta
# and lets also freeze 
def fn():
    for step in iters:

        # make a prediction
        y = model.predict(x)  # problem is that this only gives us one data point

        param_loss = model.loss(wrt=theta)
        rep_loss = g(param_loss, model.ops.theta.time, model.ops.theta.memory)
        comp_loss = f

        
        model.update_representation(loss, )
        

        
        
# slightly different framing.
loss = param + rep + comp  # <-- need a loss fn that can combine all

# must choose which one to spend compute on for the next iteration.
# maybe optimising params on data, 
# maybe decomposing params according to their structure,
# maybe improving the numerical algols

# so want to know current rate of improvement of 
# dL/dparam, dL/drep, dL/dcomp per compute spent.
# is d(dL/param)/dcompute is getting quite small 
# then maybe we should focus somewhere else.


# does this reduce back to hyperparameter optimisation!?

So we need a representation for $\theta_{\phi}$ that can be optimised.

In [7]:
# lets try a simple case, 
# learn a linear projection. 
# however the projection has 
# some sort of structure in it (some sort of TN?)
# which we would also like to learn. <-- seems hard? how!!




def loss(x, t, model):
    # theta in [n x m]
    # how can we back prop to phi?
    theta_approx = representation(theta, phi)
    
    # maybe need two copies of theta?
    # at what point can we throw away alternative representations fo theta?
    y = fn(x, theta)
    
    # weight each loss according to how much we care.
    param_loss = compare(y, t)
    rep_loss = difference(theta - theta_approx)  # this measure doesnt make sense?
    comp_loss = f(fn.ops.theta.time, model.ops.theta.memory)
    
    # want to pick phi, theta according to this loss
    # also want to choose how to spend compute?
    # finding representations vs fast kernels vs parameter settings
    return param_loss + rep_loss + comp_loss

In [ ]:
def representation(theta, phi):
    # given a set of 
    
    return theta